# PSR Factory Tutorial

### Install the most recent version from PyPI.org

In [ ]:
%pip install psr-factory

Requirements:
- Python 3.9 or greater
- Windows 10 or newer
    - Windows 7 SP1 up to Windows 8 with Visual C++ Redistributables 2015-22
- Linux, LIBC 2.28 or greater

Optional Python packages (DataFrames handling):
- numpy
- pandas
- polars


### User manual and examples

https://docs.psr-inc.com/factory/

### Downloadable versions (`pip`-less and 4.0 only currently)

https://psrenergy-docs.github.io/factory/releases.html

### Import the module to start using it

In [ ]:
import psr.factory
psr.factory.version()

### Create a case from scratch

The Factory script below will create, in memory, a case similar to Case21 found in SDDP 18 installation. 

In [ ]:
import psr.factory.samples.sddp_sample_case21 as sample_case

study = sample_case.create_sddp_sample_case21()

The code below will list all ThermalPlants attached to the study

In [ ]:
study.find("ThermalPlant")

The code below will list all Thermal plants with name starting with `T1`

In [ ]:
study.find("ThermalPlant.T1")

You can use a _glob_ pattern including wildcards such as `*` (one or more character) or `?` (one character)

In [ ]:
study.find("ThermalPlant.T*")

In [ ]:
# in this case, both will have de same result
study.find("ThermalPlant.T?")

Let's check T1

In [ ]:
t1 = study.find("ThermalPlant.T1")[0]
print(f"Code {t1.code}")
print(f"Name {t1.name}")
print(f"Type {t1.type}")

Let's traverse more plants in the case

In [ ]:
for t in study.find("ThermalPlant.*"):
    print(f"T {t.code}, {t.name}, {t.type}")

In [ ]:
for t in study.find("HydroPlant.*"):
    print(f"H {t.code}, {t.name}, {t.type}")

Lets go back to T1 and check its Installed Capacity

Properties that are not identifiers, are read using `get`, and set using... `set`

In [ ]:
t1.get("InstalledCapacity")

Improving the loop to show the initial installed capacity

In [ ]:
for t in study.find("ThermalPlant.*") + study.find("HydroPlant.*"):
    if t.type == "ThermalPlant":
        installed_capacity = t.get("InstalledCapacity")
        print(f"T {t.code}, {t.name}, {t.type}, {installed_capacity} MW")
    else:
        installed_capacity = t.get("InstalledCapacity")
        print(f"H {t.code}, {t.name}, {t.type}, {installed_capacity} MW")

Relationship between DataObject are set with special properties called `References`. A `ThermalPlant` is related to a `System` through the `RefSystem` relationship property


```mermaid
flowchart LR
    ThermalPlant -->|RefSystem| System
```

So we will read it by using `t1.get('RefSystem')`

In [ ]:
system = t1.get('RefSystem')
system

Improving our example by showing the system the plant is part of

In [ ]:
for plant in study.find("ThermalPlant") + study.find("HydroPlant"):
    system = plant.get("RefSystem")  # <-- this is how you read the related system
    print(plant.type, plant.code, plant.name, system.name)

Listing all objects in the study

In [ ]:
for obj in study.get_all_objects():
    print(obj.type, obj.code, obj.name)

#### Exploring other properties

Use `properties` method to see whats is possible to read or set. Or go to the online user manual.

In [ ]:
count = 0
for prop_name, details in t1.properties().items():
    count += 1
    print(f"Property {prop_name}")
    print(f"  Type {details.type()}")
    print()
    if count > 5:
        break

`help()` shows similar information

In [ ]:
print(t1.help())

### Creating objects

Let's create a `Fuel` DataObject and associate `T4` ThermalPlant to it.

Reminder of the relationships:

```mermaid
flowchart LR
    ThermalPlant --> |RefSystem| System
    ThermalPlant --> |RefFuels| Fuel
    Fuel --> |RefSystem| System
```

In [ ]:
fuel4 = study.create("Fuel")
fuel4.code = 4
fuel4.name = "C4"
fuel4.set("Price", 13.0)

Let's associate the fuel to a System and add it to the study

In [ ]:
system = study.find("System")[0]
fuel4.set("RefSystem", system)
study.add(fuel4)

### Copying objects

In [ ]:
t4 = t1.clone()
t4.code = 4
t4.name = "T4"
t4.set("InstalledCapacity", 20)

study.add(t4)

Associating the fuel to the ThermalPlant

In [ ]:
t4.set("RefFuels", [fuel4, ])

### Removing a DataObject from the study

In [ ]:
study.remove(t4)
study.find("ThermalPlant")

### Updating time-varying data

Let's use DataFrames to update the case demand.

The DataObject of interest is `DemandSegment`

A `DemandSegment` doesn't have a `name` or a `code`, it's related to a `Demand` object, which has name and code. One or more `DemandSegment`s can be associated with a single `Demand` to model elastic/inelastic segments.
```mermaid
flowchart LR
	Demand --> |RefSystem| System
	Demand --> |RefSegments| DemandSegment
```

This case only has one demand segment, we could access it directly. But let's access it through the main demand

In [ ]:
# Example: direct access to the unique DemandSegment in the study
demand_segment = study.find("DemandSegment")[0]
demand_segment


In [ ]:
# Example: Accessing the DemandSegment through Demand object
demand = study.find("Demand")[0]
segments = demand.get("RefSegments")
demand_segment = segments[0]
demand_segment

`get_df` function returns the property as a DataFrame. It's useful for dynamic/time-varying properties

In [ ]:
demand_segment.get_df("EnergyPerBlock")

We can use the counterpart `set_df` to pass a dataframe to the object and replace properties values

In [ ]:
doubled_df = demand_segment.get_df("EnergyPerBlock") * 2
doubled_df

In [ ]:
demand_segment.set_df(doubled_df)

Updating the dates to 2025 using some pandas kung fu

In [ ]:

demand_df = demand_segment.get_df("EnergyPerBlock")
demand_df.index = demand_df.index.map(lambda x: x.replace("2016", "2025"))
demand_df


### Saving modifications

All changes so far were made in memory. Let's write it to the disk

In [ ]:
import os
os.makedirs("my_case", exist_ok=True)
study.save("my_case")

## Exercises

### Exercise 1:

Add a modification to T4 thermal plant `InstalledCapacity` at `01/2030`. Double the current capacity.

In [ ]:
# Solution:


### Exercise 2:

Remove `Fuel` `C4` and check if it was successful (print the list of fuels)

In [ ]:
# Solution:


### Exercise 3:

Update `Study` `InitialYear` to 2025

In [ ]:
# Solution:


### Exercise 4: 

Create a `Battery` and add it to the study.
- Set its `MaximumStorage` property to 100 MWh
- Set its `MaximumCapacity` to 10 MW
- Give a nice name to it

```mermaid
flowchart LR
    Battery --> |RefSystem| System
```

In [ ]:
# Solution:
